This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

# Introduction to deep learning for computer vision

Imports

In [ ]:
from tensorflow import keras
from keras import layers

Dataset preparation

In [ ]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255

The accuracy of non-convolutional Model

In [ ]:
%%time 

tra_images = train_images.reshape(-1,28*28)
tst_images = test_images.reshape(-1,28*28)
#**************** Functional API Model **************************
input_shape = tra_images.shape[1]  # Use the actual shape of your data
inputs    = keras.Input(shape=(input_shape,))
features1 = layers.Dense(64, activation="relu")(inputs)
features2 = layers.Dense(32, activation="relu")(features1)
outputs   = layers.Dense(10, activation="softmax")(features2)
model_nc   = keras.Model(inputs, outputs)

model_nc.compile(optimizer="rmsprop",
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"])

history_nc = model_nc.fit(
    tra_images, train_labels,  # Use the correct data variable names
    epochs=10,
    batch_size=128,
    validation_split=0.2)

In [ ]:
model_nc.summary()

In [ ]:
test_loss, test_acc = model_nc.evaluate(tst_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}   .. for non Dense Model")

## Introduction to convnets
**Instantiating a small convnet**

### The convolution operation

#### Understanding border effects and padding

#### Understanding convolution strides

### The max-pooling operation

**An incorrectly structured convnet missing its max-pooling layers**

In [ ]:
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_no_max_pool = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model_no_max_pool.summary()

In [ ]:
%%time 

model_no_max_pool.compile(optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

model_no_max_pool.fit(train_images, train_labels, epochs=5, batch_size=64)

**Evaluating the convnet**


In [ ]:
test_loss, test_acc = model_no_max_pool.evaluate(test_images, test_labels)
print(f"No Maxpool Test accuracy: {test_acc:.3f}")

## Convnet with MaxPool

In [ ]:
inputs = keras.Input(shape=(28, 28, 1))

x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)

x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)

x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)

outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

**Displaying the model's summary**

In [ ]:
model.summary()

**Training the convnet on MNIST images**

In [ ]:
%%time 

model.compile(optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

model.fit(train_images, train_labels, epochs=5, batch_size=64)

**Evaluating the convnet**

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

## Model Comparison

In [ ]:
#Model           Denss          Conv-no-pool       Convolution-Maxpool
#Prameters       52,650             712,202             104,202         
#Training Time   6s                 7m14s               58s            
#Accuracy        97.3               99.1                99.2           